# Unified Bombcell Runner (Grant)

Single entrypoint replacing batch, single-probe, and NP2.0 rerun notebooks.

In [1]:
CONFIG_FILE = r'C:\Users\user\Documents\github\bombcell\py_bombcell\grant\configs\grant_recording_config_reach15_20260201_session007.json'
RUN_MODE = 'single_probe'  # batch | single_probe | np20_rerun
TARGET_PROBE = 'A'  # only used for single_probe
OVERWRITE = True


In [2]:
from pathlib import Path
import subprocess

runner = Path('run_bombcell_unified.py')
cmd = ['python', str(runner), '--config', CONFIG_FILE, '--mode', RUN_MODE]
if RUN_MODE == 'single_probe':
    cmd += ['--target-probe', TARGET_PROBE]
if OVERWRITE:
    cmd.append('--overwrite')

print('Running:', ' '.join(cmd))
subprocess.run(cmd, check=True)


Running: python run_bombcell_unified.py --config C:\Users\user\Documents\github\bombcell\py_bombcell\grant\configs\grant_recording_config_reach15_20260201_session007.json --mode single_probe --target-probe A --overwrite


CalledProcessError: Command '['python', 'run_bombcell_unified.py', '--config', 'C:\\Users\\user\\Documents\\github\\bombcell\\py_bombcell\\grant\\configs\\grant_recording_config_reach15_20260201_session007.json', '--mode', 'single_probe', '--target-probe', 'A', '--overwrite']' returned non-zero exit status 1.